# Hugging Face builder test

In [2]:
# Get the current script directory, from the notebook
import os
notebook_dir = os.getcwd()
print("Notebook directory:", notebook_dir)

model_filename = "v7-1B4"
model_file = os.path.join(notebook_dir, ".model", f"{model_filename}.pth")
print("Model file path:", model_file)

# Check if the model file exists
if os.path.isfile(model_file) is False:
    raise Exception("Model file does not exist")

# Get the project directory two levels up
project_dir = os.path.dirname(os.path.dirname(notebook_dir))
print("Project directory:", project_dir)

# Output build directory
output_dir = os.path.join(notebook_dir, f".hf_build/{model_filename}/")
print("Output directory:", output_dir)

# Empty the output directory, if it exists
if os.path.isdir(output_dir):
    import shutil
    print("Removing existing output directory")
    shutil.rmtree(output_dir)

Notebook directory: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose
Model file path: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.model/v7-1B4.pth
Project directory: /home/recursal/rwkv-prj/RWKV-block
Output directory: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.hf_build/v7-1B4/
Removing existing output directory


In [4]:
# Run the hf_builder.py
!python3 "$project_dir/hf_builder/hf_builder.py" --model_class "v7_goose" "$model_file" "$output_dir"

-----------------------------
Converting RWKV model to HuggingFace format...
Model Class     : v7_goose
Model Source    : /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.model/v7-1B4.pth
Output Directory: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.hf_build/v7-1B4/
-----------------------------
Syncing rwkv_block with HF code ...
Loading model weights raw state ...
Loading model class instance ...
Loading model state into class ...
Saving model files ...
-----------------------------
Successfully converted RWKV model to HuggingFace format
-----------------------------
